In [ ]:
import os
import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import scipy.cluster.hierarchy as spc
from sklearn.utils import resample

pio.templates.default = "plotly_white"

In [ ]:
try:
    _ = first_run
except NameError:
    first_run = True
    os.chdir(os.getcwd().rsplit("/", 1)[0])
    from _aux import functions as func

# Load data

In [ ]:
default = (
    pd.read_csv(
        "../data/train/X_train.csv",
        index_col=0,
        usecols=[
            "row_id",
            "num_arch_dc_0_12m",
            "num_arch_dc_12_24m",
            "num_arch_ok_0_12m",
            "num_arch_ok_12_24m",
            "num_arch_rem_0_12m",
            "num_arch_written_off_0_12m",
            "num_arch_written_off_12_24m",
        ],
    )
    .join(pd.read_csv("../data/train/y_train.csv", index_col=0))
    .query("default == 1")
)

not_default = (
    pd.read_csv(
        "../data/train/X_train.csv",
        index_col=0,
        usecols=[
            "row_id",
            "num_arch_dc_0_12m",
            "num_arch_dc_12_24m",
            "num_arch_ok_0_12m",
            "num_arch_ok_12_24m",
            "num_arch_rem_0_12m",
            "num_arch_written_off_0_12m",
            "num_arch_written_off_12_24m",
        ],
    )
    .join(pd.read_csv("../data/train/y_train.csv", index_col=0))
    .query("default == 0")
)

df = pd.read_csv(
    "../data/train/X_train.csv",
    index_col=0,
).join(pd.read_csv("../data/train/y_train.csv", index_col=0))

## The problematic variable

Inspite of it seeming to be a good idea to keep track of invoices that have not being paid and become loss, both variables that strive for this function are utterly problematic. Both "num_arch_written_off_0_12m" and "num_arch_written_off_12_24m" have 18% of missing values, which in itself is not a problem. The issue here lies in the fact that less than 0.1% of observations are non-zero, which means that it has an extremely low signal-to-noise ratio.


In [ ]:
df[["num_arch_written_off_0_12m", "num_arch_written_off_12_24m"]].replace(
    0, np.nan
).info()

But perhaps, those cases in which they are non-zero must be an extremely good predictor of default, right? Wrong!

In [ ]:
pd.concat(
    [
        (
            df.query("num_arch_written_off_0_12m > 0")
            .agg(
                default=("default", "sum"),
                not_default=("default", func.complement),
                count=("default", "count"),
            )
            .squeeze()
            .to_frame(name="num_arch_written_off_0_12m")
        ),
        (
            df.query("num_arch_written_off_12_24m > 0")
            .agg(
                default=("default", "sum"),
                not_default=("default", func.complement),
                count=("default", "count"),
            )
            .squeeze()
            .to_frame(name="num_arch_written_off_12_24m")
        ),
    ],
    axis=1,
).T

Therefore, we will drop these variables from our exploration

In [ ]:
default = default.drop(
    ["num_arch_written_off_0_12m", "num_arch_written_off_12_24m"], axis=1
)
not_default = not_default.drop(
    ["num_arch_written_off_0_12m", "num_arch_written_off_12_24m"], axis=1
)
df = df.drop(["num_arch_written_off_0_12m", "num_arch_written_off_12_24m"], axis=1)

## Correlation between "archived" variables

As shown in our sanity profile report, "archieved" variables have high correlation amongst themselves mainly due to overlaping lookback windows for aggregation. The first thing we must do is choose one (or some) of them to represent the group, which can be achieved by correlation clustering.

In [ ]:
corr = pd.concat([default, not_default]).corr()
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

fig = go.Figure()

fig.add_trace(
    go.Heatmap(
        z=corr.mask(mask),
        x=corr.columns,
        y=corr.columns,
        colorscale=px.colors.diverging.RdBu,
        zmin=-1,
        zmax=1,
    )
)

fig.update_layout(
    title="Correlation between 'account' variables",
    yaxis_autorange="reversed",
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    width=1000,
    height=500,
)

# fig.update_traces(opacity=0.6)
fig.show()

In [ ]:
pdist = spc.distance.pdist(corr)
linkage = spc.linkage(pdist, method="single")
idx = spc.fcluster(linkage, 0.6 * pdist.max(), "distance")

columns = [default.columns.tolist()[i] for i in list((np.argsort(idx)))]
clusterd_corr = pd.concat([default, not_default]).reindex(columns, axis=1).corr()

mask = np.zeros_like(clusterd_corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

corr["default"].to_frame(name="corr_with_label").assign(cluster=idx).drop(
    "default"
).sort_values(["cluster", "corr_with_label"], ascending=[True, False])

Let's get an overview on our clusters:

In [ ]:
pd.concat(
    [
        df[["num_arch_ok_0_12m", "num_arch_ok_12_24m"]],
        df[["num_arch_dc_0_12m", "num_arch_dc_12_24m", "num_arch_rem_0_12m"]],
    ],
    keys=["cluster_1", "cluster_2"],
    axis=1,
).describe(np.append(np.arange(0.25, 1.0, 0.1), np.array([0.99])))

## 1. "num_arch_dc_0_12m"

Despite having higher correlation with the target label, "cluster 2" variables have mostly zero values. Let's check how defaults behave across values.

In [ ]:
func.test_k_prop(
    df.groupby("num_arch_dc_0_12m").agg(
        default=("default", "sum"),
        not_default=("default", func.complement),
        count=("default", "count"),
        contamination=("default", lambda s: s.sum() / s.shape[0]),
    )
)[1]

We can observe that contamination rates increase, with some exceptions, for higher number of archived invoices of status "dc". Could we gain any benefit from transforming it into a boolean variable?

In [ ]:
(
    df.assign(var_bool=df["num_arch_dc_0_12m"] > 1)
    .groupby("var_bool")
    .agg(
        default=("default", "sum"),
        not_default=("default", func.complement),
        count=("default", "count"),
        contamination=("default", lambda s: s.sum() / s.shape[0]),
    )
)

Once again we see that transforming the variable into boolean does provide a subclass with much higher contamination rate.
We conclude that, despite having a high concentrarion of zero values, this variable (and its boolean counterpart) could be useful as features for our models.

## 2. "num_arch_ok_0_12m"

This variable has too many unique values to be considered categorical. We can take 3 different approaches to explore it: numerical, boolean and binned.

Let's first look at it as a numerical variable and verify whether its behavious for default and not_default differs significantly.


In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Histogram(
        x=not_default.num_arch_ok_0_12m.sample(1000, replace=True, random_state=42),
        name="not_default",
        histfunc="count",
        # histnorm='probability',
        xbins=dict(start=-1, end=50, size=1),
    )
)

fig.add_trace(
    go.Histogram(
        x=default.num_arch_ok_0_12m.sample(1000, replace=True, random_state=42),
        name="default",
        histfunc="count",
        # histnorm='probability',
        xbins=dict(start=-1, end=50, size=1),
    )
)

fig.update_layout(title="Title", barmode="overlay")

fig.update_traces(opacity=0.6)
fig.show()

Apparently, customers who default tend to have smaller values for "num_arch_ok_0_12m", which is in line with common sense. Users who have made successfull purchases within the last 12 months should be expected to be more reliable payers. Of course, one could lose their job, fall ill and be a victim of financial crime, all of which would impair their ability to pay their debt. Thus, this variable is not expected to perfectly explain the target label, although it is a good candidate for feature.

Let's examine whether the difference between "default" and "not_default" is indeed significant.

In [ ]:
num_iterations = 100_000
sample1 = []
sample2 = []
combined = np.concatenate(
    (default.num_arch_ok_0_12m, not_default.num_arch_ok_0_12m), axis=0
)

for i in range(num_iterations):
    np.random.seed(i)
    combined = np.concatenate(
        (
            default.num_arch_ok_0_12m.sample(1_000, replace=True),
            not_default.num_arch_ok_0_12m.sample(1_000, replace=True),
        ),
        axis=0,
    )
    sample1.append(resample(combined, n_samples=500))
    sample2.append(resample(combined, n_samples=500))

diff_bootstrap_medians = np.median(sample1, axis=1) - np.median(sample2, axis=1)

observed_difference = np.median(
    default.num_arch_ok_0_12m.sample(1_000, replace=True, random_state=42)
) - np.median(
    not_default.num_arch_ok_0_12m.sample(1_000, replace=True, random_state=42)
)

p_value = (
    diff_bootstrap_medians[diff_bootstrap_medians < observed_difference].shape[0]
    / num_iterations
)

ci_lower, ci_upper = np.percentile(diff_bootstrap_medians, [0.5, 99.5])

Due to the excessively right skewed and platykurtic distribution of "not_default" for this variable, we choose to compare medians instead of means.

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Histogram(
        x=diff_bootstrap_means,
        name="sample_difference",
        histfunc="count",
    )
)

fig.add_vline(
    x=observed_difference,
    line_width=3,
    line_color="indianred",
    line_dash="dash",
    annotation_text=f"Observed",
)

fig.add_vline(
    x=ci_lower,
    line_width=3,
    line_color="indianred",
    line_dash="solid",
    annotation_text=f"0.5%",
)

fig.add_vline(
    x=ci_upper,
    line_width=3,
    line_color="indianred",
    line_dash="solid",
    annotation_text=f"99.5%",
)

fig.update_layout(
    title="Distribution of difference in medians",
    barmode="overlay",
)

fig.update_traces(opacity=0.75)
# fig.update_xaxes(range=[-5, 5])
fig.show()

This result shows us that there is a significative difference in medians, which in turn suggests that this variable could be useful for model induction. Now, we analyze whether its binned counterpart is too a good candidate.

In [ ]:
(
    df[["default"]]
    .assign(
        bins_var=pd.cut(df["num_arch_ok_0_12m"], 10),
    )
    .groupby("bins_var")
    .agg(
        default=("default", "sum"),
        not_default=("default", func.complement),
        count=("default", "count"),
        contamination=("default", lambda s: s.sum() / s.shape[0]),
    )
)

The fact that using 10 bins has very little effect on the contamination rate shows that this is not a good strategy to move forward with. Hopefully, its boolean counterpart can perform better.

In [ ]:
(
    df[["default"]]
    .assign(
        bool_var=df["num_arch_ok_0_12m"] > 1,
    )
    .groupby("bool_var")
    .agg(
        default=("default", "sum"),
        not_default=("default", func.complement),
        count=("default", "count"),
        contamination=("default", lambda s: s.sum() / s.shape[0]),
    )
)

The boolean variable does increase contamination in 2 fold for the group "up to 1" at the expense of a very diluted rate for the "above 1" group, which wouldn't be a problem if the latter held few "default" observations. However, the group holds almost 20% of "default" observations, thus making the boolean variable also unsuitable.

--

To sum up, we show that "num_arch_dc_0_12m" and its boolean counterpart are good candidates for features.
Furthermore, we test three variations of "num_arch_ok_0_12m", which showed to be best in its original form.
Hence, we move forward with 3 candidates for features from "archieved" variables:
- num_arch_dc_0_12m
- num_arch_dc_0_12m_is_above_1
- num_arch_ok_0_12m

Next, we look at "order" variables.